##requiremnts


In [2]:
pip install SimpleITK

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 13.0 MB/s eta 0:00:00


In [3]:
pip install scipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
pip install torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
pip install evalutils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 551.2/551.2 kB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 10.7 MB/s eta 0:00:00


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import zipfile



DATASET_ZIP_DIR ='/content/drive/MyDrive/training_utils (1).zip' # Setting the File location for testing data. Here it should be set to the file location of BN-HTRD_BigData.zip 

with zipfile.ZipFile(DATASET_ZIP_DIR,"r") as zip_ref:
    zip_ref.extractall("/content")
%cd /content/training_utils

/content/training_utils


In [8]:
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

## model class

In [9]:
import SimpleITK
import numpy as np

from pandas import DataFrame
from scipy.ndimage import center_of_mass, label
import torchvision
import torchvision
from torchvision import transforms 
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torch
from evalutils import DetectionAlgorithm
from evalutils.validators import (
    UniquePathIndicesValidator,
    UniqueImagesValidator,
)
from skimage import transform
import json
from typing import Dict
import training_utils.utils as utils
import training_utils.utils_ObjectDetection as utils2
from training_utils.dataset import CXRNoduleDataset, get_transform
import os
from training_utils.train import train_one_epoch
import itertools
from pathlib import Path
from postprocessing import get_NonMaxSup_boxes

'''
NODE21 template nodule detection codebase
Author: Ecem Sogancioglu
email: ecemsogancioglu@gmail.com
'''

# This parameter adapts the paths between local execution and execution in docker. You can use this flag to switch between these two modes.
# For building your docker, set this parameter to True. If False, it will run process.py locally for test purposes.
execute_in_docker = True

class Noduledetection(DetectionAlgorithm):
    def __init__(self, input_dir, output_dir, train=False, retrain=False, retest=False):
        super().__init__(
            validators=dict(
                input_image=(
                    UniqueImagesValidator(),
                    UniquePathIndicesValidator(),
                )
            ),
            input_path = Path(input_dir),
            output_path = Path(output_dir),
            output_file = Path(os.path.join(output_dir,'nodules.json'))
        )
        
        #------------------------------- LOAD the model here ---------------------------------
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.input_path, self.output_path = input_dir, output_dir
        print('using the device ', self.device)
        self.model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights=None, weights_backbone=None)
        num_classes = 2  # 1 class (nodule) + background
        in_features = self.model.roi_heads.box_predictor.cls_score.in_features
        self.model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
        self.transforms = transforms
        if not (train or retest):
            # retrain or test phase
            print('loading the model.pth file :')
            self.model.load_state_dict(
            torch.load(
                Path(os.path.join(self.output_path,'model_retrained.pth')) if execute_in_docker else Path("model.pth"),
                map_location=self.device,
                )
            ) 
            
        if retest:
            print('loading the retrained model_retrained.pth file')
            self.model.load_state_dict(
            torch.load(
                Path(os.path.join(self.input_path,'model_retrained.pth')),
                map_location=self.device,
                )
            ) 
            
        self.model.to(self.device)
        
    def save(self):
        with open(str(self._output_file), "w") as f:
            json.dump(self._case_results[0], f)
            
    # TODO: Copy this function for your processor as well!
    def process_case(self, *, idx, case):
        '''
        Read the input, perform model prediction and return the results. 
        The returned value will be saved as nodules.json by evalutils.
        process_case method of evalutils
        (https://github.com/comic/evalutils/blob/fd791e0f1715d78b3766ac613371c447607e411d/evalutils/evalutils.py#L225) 
        is overwritten here, so that it directly returns the predictions without changing the format.
        
        '''
        # Load and test the image for this case
        input_image, input_image_file_path = self._load_input_image(case=case)
        
        # Detect and score candidates
        scored_candidates = self.predict(input_image=input_image)
        
        # Write resulting candidates to nodules.json for this case
        return scored_candidates
    
   #--------------------Write your retrain function here ------------
    def make_prediction(self,model, img, threshold):
      
      self.model.eval()
      #img=self.transforms(img)
      #image_data = SimpleITK.GetArrayFromImage(img)
      #spacing = img.GetSpacing()
     # image_data = np.array(img)
        
      # if len(image_data.shape)==2:
      #     image_data = np.expand_dims(image_data, 0)
      preds = model(img)
      for id in range(len(preds)) :
          idx_list = []

          for idx, score in enumerate(preds[id]['scores']) :
              if score > threshold : 
                  idx_list.append(idx)

          preds[id]['boxes'] = preds[id]['boxes'][idx_list]
          preds[id]['labels'] = preds[id]['labels'][idx_list]
          preds[id]['scores'] = preds[id]['scores'][idx_list]

          return preds 
          
    #--------------------Write your retrain function here ------------
    def train(self, num_epochs = 64):
        '''
        input_dir: Input directory containing all the images to train with
        output_dir: output_dir to write model to.
        num_epochs: Number of epochs for training the algorithm.
        '''
        # Implementation of the pytorch model and training functions is based on pytorch tutorial: https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html

        # create training dataset and defined transformations
        self.model.train() 
        input_dir = self.input_path
        dataset = CXRNoduleDataset(input_dir, os.path.join(input_dir, 'metadata.csv'), get_transform(train=True))
        test_dataset = CXRNoduleDataset(output_dir, os.path.join(output_dir, 'metadata.csv'), get_transform(train=True))
        print('training starts ')
        # define training and validation data loaders
        data_loader = torch.utils.data.DataLoader(
            dataset, batch_size=6, shuffle=True, num_workers=2,
            collate_fn=utils.collate_fn)
        test_loader = torch.utils.data.DataLoader(
            test_dataset, batch_size=6, shuffle=True, num_workers=2,
            collate_fn=utils.collate_fn)
    
        # construct an optimizer
        params = [p for p in self.model.parameters() if p.requires_grad]
        optimizer = torch.optim.SGD(params, lr=0.005,
                                    momentum=0.9, weight_decay=0.0005)
        # and a learning rate scheduler
        lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                       step_size=3,
                                                       gamma=0.1)        
        for epoch in range(num_epochs):
            train_one_epoch(self.model, optimizer, data_loader, self.device, epoch, print_freq=2442)
            # update the learning rate
            lr_scheduler.step()
            print('epoch ', str(epoch),' is running')
            # evaluate on the test dataset
            
            #IMPORTANT: save retrained version frequently.
            print('saving the model')
            torch.save(self.model.state_dict(), os.path.join(self.output_path, 'model_retrained.pth'))
            ##sth new
            print('loading the model.pth file :')
            self.model.load_state_dict(torch.load(os.path.join(self.output_path, 'model_retrained.pth') if execute_in_docker else Path("model.pth"),map_location=self.device,))

            from tqdm import tqdm

            labels = []
            preds_adj_all = []
            annot_all = []
            device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu') 

            for im, target,image_namee in tqdm(test_loader, position = 0, leave = True):
              im = list(img.to(device) for img in im)
              #im = (img.to(device) for img in im)
              #im = to_device(im, device)

              #annot = [{k: v.to(device) for k, v in t.items()} for t in annot]

              for t in target:
                #print(t['labels'])
                
                labels += t['labels']

                with torch.no_grad():
                  preds_adj = self.make_prediction(self.model, im, 0.38)
                  #preds_adj = self.predict(input_image=im)
                  preds_adj = [{k: v.to(torch.device('cpu')) for k, v in t.items()} for t in preds_adj]
                  preds_adj_all.append(preds_adj)
                  annot_all.append(target)
            sample_metrics = []
            for batch_i in range(len(preds_adj_all)):
              sample_metrics += utils2.get_batch_statistics(preds_adj_all[batch_i], annot_all[batch_i], iou_threshold=0.5) 

            true_positives, pred_scores, pred_labels = [torch.cat(x, 0) for x in list(zip(*sample_metrics))]  # 배치가 전부 합쳐짐
            precision, recall, AP, f1, ap_class = utils2.ap_per_class(true_positives, pred_scores, pred_labels, torch.tensor(labels))
            APF=AP.type(torch.FloatTensor)
            mAP = torch.mean(APF)
            print('\n')
            print(f'mAP : {mAP}')
            print(f'AP : {AP}') 
            print(f'Precision : {precision}')  
            print(f'recall : {recall}') 
            print(f'f1 : {f1}')
            print(f'ap_class : {ap_class}')   

      

    def format_to_GC(self, np_prediction, spacing) -> Dict:
        '''
        Convenient function returns detection prediction in required grand-challenge format.
        See:
        https://comic.github.io/grandchallenge.org/components.html#grandchallenge.components.models.InterfaceKind.interface_type_annotation
        
        
        np_prediction: dictionary with keys boxes and scores.
        np_prediction[boxes] holds coordinates in the format as x1,y1,x2,y2
        spacing :  pixel spacing for x and y coordinates.
        
        return:
        a Dict in line with grand-challenge.org format.
        '''
        # For the test set, we expect the coordinates in millimeters. 
        # this transformation ensures that the pixel coordinates are transformed to mm.
        # and boxes coordinates saved according to grand challenge ordering.
        x_y_spacing = [spacing[0], spacing[1], spacing[0], spacing[1]]
        boxes = []
        for i, bb in enumerate(np_prediction['boxes']):
            box = {}   
            box['corners']=[]
            x_min, y_min, x_max, y_max = bb*x_y_spacing
            x_min, y_min, x_max, y_max  = round(x_min, 2), round(y_min, 2), round(x_max, 2), round(y_max, 2)
            bottom_left = [x_min, y_min,  np_prediction['slice'][i]] 
            bottom_right = [x_max, y_min,  np_prediction['slice'][i]]
            top_left = [x_min, y_max,  np_prediction['slice'][i]]
            top_right = [x_max, y_max,  np_prediction['slice'][i]]
            box['corners'].extend([top_right, top_left, bottom_left, bottom_right])
            box['probability'] = round(float(np_prediction['scores'][i]), 2)
            boxes.append(box)
        
        return dict(type="Multiple 2D bounding boxes", boxes=boxes, version={ "major": 1, "minor": 0 })
        
    def merge_dict(self, results):
        merged_d = {}
        for k in results[0].keys():
            merged_d[k] = list(itertools.chain(*[d[k] for d in results]))
        return merged_d
        
    def predict(self, *, input_image: SimpleITK.Image) -> DataFrame:
        self.model.eval() 
        
        image_data = SimpleITK.GetArrayFromImage(input_image)
        spacing = input_image.GetSpacing()
        image_data = np.array(image_data)
        
        if len(image_data.shape)==2:
            image_data = np.expand_dims(image_data, 0)
            
        results = []
        # operate on 3D image (CXRs are stacked together)
        for j in range(len(image_data)):
            # Pre-process the image
            image = image_data[j,:,:]
            # The range should be from 0 to 1.
            image = image.astype(np.float32) / np.max(image)  # normalize
            image = np.expand_dims(image, axis=0)
            tensor_image = torch.from_numpy(image).to(self.device)#.reshape(1, 1024, 1024)
            with torch.no_grad():
                prediction = self.model([tensor_image.to(self.device)])
            
            prediction = [get_NonMaxSup_boxes(prediction[0])]
            # convert predictions from tensor to numpy array.
            np_prediction = {str(key):[i.cpu().numpy() for i in val]
                   for key, val in prediction[0].items()}
            np_prediction['slice'] = len(np_prediction['boxes'])*[j]
            results.append(np_prediction)
        
        predictions = self.merge_dict(results)
        data = self.format_to_GC(predictions, spacing)
        print(data)
        return data

# if __name__ == "__main__":
#     import argparse
#     parser = argparse.ArgumentParser(
#         prog='process.py',
#         description=
#             'Reads all images from an input directory and produces '
#             'results in an output directory')

#     parser.add_argument('input_dir', help = "input directory to process")
#     parser.add_argument('output_dir', help = "output directory generate result files in")
#     parser.add_argument('--train', action='store_true', help = "Algorithm on train mode.")
#     parser.add_argument('--retrain', action='store_true', help = "Algorithm on retrain mode (loading previous weights).")
#     parser.add_argument('--retest', action='store_true', help = "Algorithm on evaluate mode after retraining.")

#     parsed_args = parser.parse_args()  
#     if (parsed_args.train or parsed_args.retrain):# train mode: retrain or train
#         Noduledetection(parsed_args.input_dir, parsed_args.output_dir, parsed_args.train, parsed_args.retrain, parsed_args.retest).train()
#     else:# test mode (test or retest)
#         Noduledetection(parsed_args.input_dir, parsed_args.output_dir, retest=parsed_args.retest).process()

In [ ]:

# greet = "Hello World!"

# print(greet[0:6])
# h=[]
# g={
#     "boxes": '0.0',
#     "labels" : '0',
#     "image_id" : '0.0',
#     "area": '0.0',
#     "iscrowd" : '0.0',

# }
# h += g["labels" ]
# print(g["labels" ])
# print(h)
# boxes=0
# labels=0
# image_id=0
# area=0
# iscrowd=0
# hh=[]
# target = {}
# target["boxes"] = boxes
# target["labels"] = labels
# target["image_id"] = image_id
# target["area"] = area
# target["iscrowd"] = iscrowd
# print(target)
# hh += 0
# print(hh)

##train

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
import zipfile



DATASET_ZIP_DIR ='/content/drive/MyDrive/train_data.zip' # Setting the File location for testing data. Here it should be set to the file location of BN-HTRD_BigData.zip 

with zipfile.ZipFile(DATASET_ZIP_DIR,"r") as zip_ref:
    zip_ref.extractall("/content/data")

In [ ]:
input_dir='/content/data/train_data'
output_dir='/content/drive/MyDrive/output'
aa=Noduledetection( input_dir, output_dir, train=True, retrain=False, retest=False)
aa.train()

using the device  cuda
training starts 
Epoch: [0]  [  0/333]  eta: 1:05:15  lr: 0.000020  loss: 1.6118 (1.6118)  loss_classifier: 0.9133 (0.9133)  loss_box_reg: 0.0008 (0.0008)  loss_objectness: 0.6916 (0.6916)  loss_rpn_box_reg: 0.0061 (0.0061)  time: 11.7572  data: 0.6686  max mem: 8947
Epoch: [0]  [332/333]  eta: 0:00:01  lr: 0.005000  loss: 0.0705 (0.1462)  loss_classifier: 0.0209 (0.0324)  loss_box_reg: 0.0068 (0.0036)  loss_objectness: 0.0433 (0.1046)  loss_rpn_box_reg: 0.0040 (0.0056)  time: 1.3845  data: 0.0185  max mem: 9108
Epoch: [0] Total time: 0:07:56 (1.4312 s / it)
epoch  0  is running
saving the model
loading the model.pth file :


100%|██████████| 36/36 [01:51<00:00,  3.11s/it]




mAP : 0.0
AP : tensor([0])
Precision : tensor([0])
recall : tensor([0])
f1 : tensor([0.])
ap_class : tensor([1])
Epoch: [1]  [  0/333]  eta: 0:10:31  lr: 0.005000  loss: 0.0596 (0.0596)  loss_classifier: 0.0173 (0.0173)  loss_box_reg: 0.0079 (0.0079)  loss_objectness: 0.0318 (0.0318)  loss_rpn_box_reg: 0.0026 (0.0026)  time: 1.8957  data: 0.4699  max mem: 9111


KeyboardInterrupt: ignored

In [ ]:
input_dir='/content/data/train_data'
output_dir='/content/drive/MyDrive/output'
aa=Noduledetection( input_dir, output_dir, train=False, retrain=True, retest=False)
aa.train()

using the device  cuda
loading the model.pth file :
training starts 
Epoch: [0]  [  0/333]  eta: 0:55:19  lr: 0.000020  loss: 0.1063 (0.1063)  loss_classifier: 0.0464 (0.0464)  loss_box_reg: 0.0476 (0.0476)  loss_objectness: 0.0108 (0.0108)  loss_rpn_box_reg: 0.0015 (0.0015)  time: 9.9694  data: 0.5403  max mem: 8947
Epoch: [0]  [332/333]  eta: 0:00:01  lr: 0.005000  loss: 0.0601 (0.0802)  loss_classifier: 0.0281 (0.0315)  loss_box_reg: 0.0230 (0.0333)  loss_objectness: 0.0105 (0.0123)  loss_rpn_box_reg: 0.0021 (0.0030)  time: 1.3388  data: 0.0181  max mem: 9108
Epoch: [0] Total time: 0:07:38 (1.3762 s / it)
epoch  0  is running
saving the model
loading the model.pth file :


100%|██████████| 36/36 [02:13<00:00,  3.72s/it]




mAP : 1.90220308303833
AP : tensor([1.9022], dtype=torch.float64)
Precision : tensor([0.1060])
recall : tensor([2.8750])
f1 : tensor([0.2044])
ap_class : tensor([1])
Epoch: [1]  [  0/333]  eta: 0:11:30  lr: 0.005000  loss: 0.0607 (0.0607)  loss_classifier: 0.0292 (0.0292)  loss_box_reg: 0.0197 (0.0197)  loss_objectness: 0.0109 (0.0109)  loss_rpn_box_reg: 0.0009 (0.0009)  time: 2.0722  data: 0.6299  max mem: 9110
Epoch: [1]  [332/333]  eta: 0:00:01  lr: 0.005000  loss: 0.0862 (0.0849)  loss_classifier: 0.0334 (0.0341)  loss_box_reg: 0.0321 (0.0339)  loss_objectness: 0.0103 (0.0136)  loss_rpn_box_reg: 0.0024 (0.0033)  time: 1.3452  data: 0.0191  max mem: 9112
Epoch: [1] Total time: 0:07:33 (1.3606 s / it)
epoch  1  is running
saving the model
loading the model.pth file :


100%|██████████| 36/36 [01:46<00:00,  2.95s/it]




mAP : 2.280296564102173
AP : tensor([2.2803], dtype=torch.float64)
Precision : tensor([0.0666])
recall : tensor([3.8403])
f1 : tensor([0.1310])
ap_class : tensor([1])
Epoch: [2]  [  0/333]  eta: 0:12:32  lr: 0.005000  loss: 0.0378 (0.0378)  loss_classifier: 0.0135 (0.0135)  loss_box_reg: 0.0170 (0.0170)  loss_objectness: 0.0067 (0.0067)  loss_rpn_box_reg: 0.0005 (0.0005)  time: 2.2596  data: 0.7227  max mem: 9112
Epoch: [2]  [332/333]  eta: 0:00:01  lr: 0.005000  loss: 0.0623 (0.0833)  loss_classifier: 0.0278 (0.0335)  loss_box_reg: 0.0229 (0.0342)  loss_objectness: 0.0100 (0.0123)  loss_rpn_box_reg: 0.0015 (0.0032)  time: 1.3483  data: 0.0197  max mem: 9112
Epoch: [2] Total time: 0:07:34 (1.3634 s / it)
epoch  2  is running
saving the model
loading the model.pth file :


...



mAP : 2.739612340927124
AP : tensor([2.7396], dtype=torch.float64)
Precision : tensor([0.0742])
recall : tensor([4.2917])
f1 : tensor([0.1459])
ap_class : tensor([1])
Epoch: [5]  [  0/333]  eta: 0:10:03  lr: 0.000500  loss: 0.0446 (0.0446)  loss_classifier: 0.0264 (0.0264)  loss_box_reg: 0.0123 (0.0123)  loss_objectness: 0.0053 (0.0053)  loss_rpn_box_reg: 0.0006 (0.0006)  time: 1.8122  data: 0.4322  max mem: 9113
Epoch: [5]  [332/333]  eta: 0:00:01  lr: 0.000500  loss: 0.0566 (0.0750)  loss_classifier: 0.0229 (0.0290)  loss_box_reg: 0.0311 (0.0347)  loss_objectness: 0.0034 (0.0086)  loss_rpn_box_reg: 0.0015 (0.0027)  time: 1.3512  data: 0.0214  max mem: 9113
Epoch: [5] Total time: 0:07:33 (1.3619 s / it)
epoch  5  is running
saving the model
loading the model.pth file :


100%|██████████| 36/36 [01:45<00:00,  2.93s/it]




mAP : 2.860989809036255
AP : tensor([2.8610], dtype=torch.float64)
Precision : tensor([0.0814])
recall : tensor([4.5833])
f1 : tensor([0.1600])
ap_class : tensor([1])
Epoch: [6]  [  0/333]  eta: 0:10:08  lr: 0.000050  loss: 0.0160 (0.0160)  loss_classifier: 0.0082 (0.0082)  loss_box_reg: 0.0051 (0.0051)  loss_objectness: 0.0025 (0.0025)  loss_rpn_box_reg: 0.0001 (0.0001)  time: 1.8287  data: 0.4660  max mem: 9113
Epoch: [6]  [332/333]  eta: 0:00:01  lr: 0.000050  loss: 0.0582 (0.0741)  loss_classifier: 0.0290 (0.0289)  loss_box_reg: 0.0275 (0.0348)  loss_objectness: 0.0071 (0.0076)  loss_rpn_box_reg: 0.0019 (0.0027)  time: 1.3457  data: 0.0208  max mem: 9113
Epoch: [6] Total time: 0:07:33 (1.3616 s / it)
epoch  6  is running
saving the model
loading the model.pth file :


100%|██████████| 36/36 [01:45<00:00,  2.93s/it]




mAP : 2.9498047828674316
AP : tensor([2.9498], dtype=torch.float64)
Precision : tensor([0.0802])
recall : tensor([4.4167])
f1 : tensor([0.1576])
ap_class : tensor([1])
Epoch: [7]  [  0/333]  eta: 0:10:00  lr: 0.000050  loss: 0.0890 (0.0890)  loss_classifier: 0.0315 (0.0315)  loss_box_reg: 0.0415 (0.0415)  loss_objectness: 0.0125 (0.0125)  loss_rpn_box_reg: 0.0035 (0.0035)  time: 1.8037  data: 0.4415  max mem: 9113


In [ ]:
function ClickConnect(){
console.log("Working"); 
document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();
}
setInterval(ClickConnect,60000)

SyntaxError: ignored

In [ ]:
a = torch.tensor([1,2])

a.dtype
#torch.int64

a.type(torch.FloatTensor)
#tensor([1., 2.])

tensor([1., 2.])

In [ ]:
x=[1024, 1024]
x=torch.Tensor(x)
print(x.shape)
y = x.unsqueeze(0) # Add new dimension at position 0
print(y.shape)

torch.Size([2])
torch.Size([1, 2])


##score


In [ ]:
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False, pretrained_backbone=False)
num_classes = 2  # 1 class (nodule) + background
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

model.load_state_dict(torch.load('/content/drive/MyDrive/output/model_retrained.pth'))#'/content/drive/MyDrive/output/model_retrained.pth'))
model.eval()

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       

In [ ]:
def get_model_instance_segmentation(num_classes):
  
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn()
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    return model

In [ ]:
# model = get_model_instance_segmentation(2)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu') 
model.to(device)

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       

In [ ]:
# state_dict = torch.load('/content/drive/MyDrive/faster/Faster_RCNN_model_62.pth',map_location=torch.device('cpu'))
# model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False, pretrained_backbone=False)

# model.load_state_dict(state_dict)
torch.load('/content/drive/MyDrive/output/model_retrained.pth',map_location=torch.device('cpu'))

OrderedDict([('backbone.body.conv1.weight',
              tensor([[[[ 2.1814e-02,  4.0958e-02,  7.1677e-03,  ...,  1.9445e-02,
                         -1.6082e-03, -8.1562e-03],
                        [ 2.8557e-02,  6.2053e-03,  4.1019e-02,  ...,  5.3659e-02,
                          2.0352e-02,  3.0972e-02],
                        [ 1.3953e-02,  1.0178e-02,  3.9927e-03,  ...,  2.4699e-02,
                          3.6262e-02, -1.3329e-02],
                        ...,
                        [ 2.4810e-02, -4.1146e-02,  1.8433e-02,  ...,  1.5586e-02,
                          1.3648e-02,  4.4812e-02],
                        [-1.3096e-02,  2.1951e-02, -7.1533e-03,  ...,  3.9758e-02,
                         -6.8637e-03, -5.1230e-03],
                        [-9.2286e-03,  4.0579e-02,  1.5314e-02,  ...,  2.3155e-02,
                         -4.1546e-02,  2.0363e-02]],
              
                       [[ 2.8401e-02, -2.5868e-03,  6.7304e-02,  ...,  3.5458e-02,
                  

In [ ]:
%cd /content/training_utils
from training_utils import utils_ObjectDetection as utils2

/content/training_utils


In [ ]:
import SimpleITK
import numpy as np

from pandas import DataFrame
from scipy.ndimage import center_of_mass, label
import torchvision
import torchvision
from torchvision import transforms 
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torch
from evalutils import DetectionAlgorithm
from evalutils.validators import (
    UniquePathIndicesValidator,
    UniqueImagesValidator,
)
from skimage import transform
import json
from typing import Dict
import training_utils.utils as utils
import training_utils.utils_ObjectDetection as utils2
from training_utils.dataset import CXRNoduleDataset, get_transform
import os
from training_utils.train import train_one_epoch
import itertools
from pathlib import Path
from postprocessing import get_NonMaxSup_boxes

In [ ]:
def make_prediction(model, img, threshold):
      
  model.eval()
      #img=self.transforms(img)
      #image_data = SimpleITK.GetArrayFromImage(img)
      #spacing = img.GetSpacing()
     # image_data = np.array(img)
        
      # if len(image_data.shape)==2:
      #     image_data = np.expand_dims(image_data, 0)
  preds = model(img)
  #print('preds',preds)
  for id in range(len(preds)) :
    idx_list = []

    for idx, score in enumerate(preds[id]['scores']) :
      if score > threshold : 
        idx_list.append(idx)

    preds[id]['boxes'] = preds[id]['boxes'][idx_list]
    preds[id]['labels'] = preds[id]['labels'][idx_list]
    preds[id]['scores'] = preds[id]['scores'][idx_list]
  print('preds',preds)
  return preds

In [ ]:
from training_utils.dataset import CXRNoduleDataset, get_transform
output_dir='/content/drive/MyDrive/output'
test_dataset = CXRNoduleDataset(output_dir, os.path.join(output_dir, 'metadata.csv'), get_transform(train=True))
test_loader = torch.utils.data.DataLoader(
            test_dataset, batch_size=6, shuffle=True, num_workers=2,
            collate_fn=utils.collate_fn)

In [ ]:
from tqdm import tqdm

labels = []
preds_adj_all = []
annot_all = []
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu') 

for im, target,image_namee in tqdm(test_loader, position = 0, leave = True):

  im = list(img.to(device) for img in im)
              #im = (img.to(device) for img in im)
  #im = to_device(im, device)

              #annot = [{k: v.to(device) for k, v in t.items()} for t in annot]

  for t in target:
                #print(t['labels'])
                
    labels += t['labels']
    #print(labels)

  with torch.no_grad():
    preds_adj = make_prediction(model, im, 0.001)
    print('preds_adj',preds_adj)
                    
    preds_adj = [{k: v.to(torch.device('cpu')) for k, v in t.items()} for t in preds_adj]
    preds_adj_all.append(preds_adj)
    annot_all.append(target)
print('ann_all',annot_all[0])
sample_metrics = []
for batch_i in range(len(preds_adj_all)):
  sample_metrics += utils2.get_batch_statistics(preds_adj_all[batch_i], annot_all[batch_i], iou_threshold=0.5) 

true_positives, pred_scores, pred_labels = [torch.cat(x, 0) for x in list(zip(*sample_metrics))]  # 배치가 전부 합쳐짐
precision, recall, AP, f1, ap_class = utils2.ap_per_class(true_positives, pred_scores, pred_labels, torch.tensor(labels))
APF=AP.type(torch.FloatTensor)
mAP = torch.mean(APF)
print('\n')
print(f'mAP : {mAP}')
print(f'AP : {AP}') 
print(f'Precision : {precision}')  
print(f'recall : {recall}') 
print(f'f1 : {f1}')
print(f'ap_class : {ap_class}')  

In [ ]:
%cd /content/training_utils
from training_utils import utils_ObjectDetection as utils

/content/training_utils


In [ ]:
sample_metrics = []
for batch_i in range(len(preds_adj_all)):
    sample_metrics += utils.get_batch_statistics(preds_adj_all[batch_i], annot_all[batch_i], iou_threshold=0.5) 

true_positives, pred_scores, pred_labels = [torch.cat(x, 0) for x in list(zip(*sample_metrics))]  # 배치가 전부 합쳐짐
precision, recall, AP, f1, ap_class = utils.ap_per_class(true_positives, pred_scores, pred_labels, torch.tensor(labels))
mAP = torch.mean(AP)
print(f'mAP : {mAP}')
print(f'AP : {AP}')
